## Import modules and create Spark session

In [1]:
#import module
from pyspark.ml.recommendation import ALS

#create session
appName = "Recommender system in Spark"
spark = SparkSession \
    .builder \
    .appName(appName) \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

## Read file into dataFrame

In [2]:
#read file into dataFrame using automatically inferred schema
ratings = spark.read.csv('dataset/ratings.csv', inferSchema=True, header=True)
movies = spark.read.csv('dataset/movies.csv', inferSchema=True, header=True)
#merge "movies" and "ratings" dataFrame based on "movieId"
ratings.join(movies, "movieId").show(3)

+-------+------+------+----------+--------------------+--------------------+
|movieId|userId|rating| timestamp|               title|              genres|
+-------+------+------+----------+--------------------+--------------------+
|     31|     1|   2.5|1260759144|Dangerous Minds (...|               Drama|
|   1029|     1|   3.0|1260759179|        Dumbo (1941)|Animation|Childre...|
|   1061|     1|   3.0|1260759182|     Sleepers (1996)|            Thriller|
+-------+------+------+----------+--------------------+--------------------+
only showing top 3 rows



## Data preparation

In [3]:
#use only column data of "userId", "movieId", dan "rating"
data = ratings.select("userId", "movieId", "rating")
#divide data, 70% for training and 30% for testing
splits = data.randomSplit([0.7, 0.3])
train = splits[0].withColumnRenamed("rating", "label")
test = splits[1].withColumnRenamed("rating", "trueLabel")
#calculate number of rows
train_rows = train.count()
test_rows = test.count()
print ("number of training data rows:", train_rows, 
       ", number of testing data rows:", test_rows)

number of training data rows: 69978 , number of testing data rows: 30026


## Define model and train it

In [5]:
#define ALS (Alternating Least Square) as our recommender system
als = ALS(maxIter=19, regParam=0.01, userCol="userId", 
          itemCol="movieId", ratingCol="label")
#train our ALS model
model = als.fit(train)
print("Training is done!")

Training is done!


## Predict testing data

In [14]:
prediction = model.transform(test)
print("testing is done!")

testing is done!


In [19]:
prediction.join(movies, "movieId").select(
    "userId", "title", "prediction", "trueLabel").show(n=10, truncate=False)

+------+---------------------------+----------+---------+
|userId|title                      |prediction|trueLabel|
+------+---------------------------+----------+---------+
|242   |Guilty as Sin (1993)       |3.4961054 |4.0      |
|126   |Hudsucker Proxy, The (1994)|4.166239  |5.0      |
|285   |Hudsucker Proxy, The (1994)|3.314852  |5.0      |
|274   |Hudsucker Proxy, The (1994)|4.180437  |5.0      |
|452   |Hudsucker Proxy, The (1994)|2.7477295 |3.0      |
|15    |Hudsucker Proxy, The (1994)|2.4086752 |3.0      |
|358   |Hudsucker Proxy, The (1994)|3.280146  |5.0      |
|659   |Hudsucker Proxy, The (1994)|3.18338   |4.0      |
|537   |Hudsucker Proxy, The (1994)|4.264035  |5.0      |
|241   |Hudsucker Proxy, The (1994)|3.7120972 |4.0      |
+------+---------------------------+----------+---------+
only showing top 10 rows



## Evaluate the accuracy of our model

In [20]:
#import RegressionEvaluator since we also want to calculate RMSE (Root Mean Square Error)
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(
    labelCol="trueLabel", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(prediction)
print ("Root Mean Square Error (RMSE):", rmse)

Root Mean Square Error (RMSE): nan


In [21]:
prediction.count()
a = prediction.count()
print("number of original data rows: ", a)
#drop rows with any missing data
cleanPred = prediction.dropna(how="any", subset=["prediction"])
b = cleanPred.count()
print("number of rows after dropping data with missing value: ", b)
print("number of missing data: ", a-b)

number of original data rows:  30026
number of rows after dropping data with missing value:  28826
number of missing data:  1200


In [22]:
rmse = evaluator.evaluate(cleanPred)
print ("Root Mean Square Error (RMSE):", rmse)

Root Mean Square Error (RMSE): 1.2398830770288136
